### Quick script to get gene or CDS sequences from gff file

In [1]:
import pandas as pd
import pybedtools 
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import collections

In [2]:
in_dir = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
feature = "CDS"

In [3]:
def col_8_id(x):
    import re
    pattern = r'ID=([a-zA-Z0-9_.]*)'
    regex = re.compile(pattern)
    m = regex.search(x)
    match = m.groups()[0].replace('TU', 'model')
    if match.startswith('cds.'):
        match = match[4:]
    if 'exon' in match:
        _list = match.split('.')
        match = '.'.join(_list[:-1])
    return match

In [16]:
anno_gff = [x for x in os.listdir(in_dir) if x.endswith('anno.gff3')]
anno_gff.sort()
genome_fa = [x for x in os.listdir(in_dir) if x.endswith('_ctg.fa')]
genome_fa.sort()
if len(anno_gff) != len(genome_fa):
    print('Check if all anno.gff3 and genome fa files are available. It should be three each!')
print(genome_fa)
print(anno_gff)

['Pst_104E_v12_h_ctg.fa', 'Pst_104E_v12_p_ctg.fa', 'Pst_104E_v12_ph_ctg.fa']
['Pst_104E_v12_h_ctg.anno.gff3', 'Pst_104E_v12_p_ctg.anno.gff3', 'Pst_104E_v12_ph_ctg.anno.gff3']


In [30]:
if feature == 'CDS':
    for i, gff in enumerate(anno_gff):
        print(gff)
        gff_df = pd.read_csv(in_dir+'/'+gff, sep='\t', header=None)
        gff_df_CDS = gff_df[gff_df[2].str.contains(feature)]
        gff_df_CDS[2] = gff_df_CDS[8].apply(col_8_id)
        out_fn = gff.replace('anno', 'anno.'+feature)
        gff_df_CDS.to_csv(in_dir+'/'+out_fn, sep='\t', header=None, index=None)
        #here generate a gff where the name column (==3) contains the ID of the CDS.start_end_phase_strand
        #this will be used later to pull out the information after bedtools was run
        gff_df_CDS[2] = gff_df_CDS[2]+'.'+gff_df_CDS[3].astype(str)+'_'+gff_df_CDS[4].astype(str)+'_'+gff_df_CDS[6].astype(str)+'_'+gff_df_CDS[7].astype(str) 
        out_fn += '.tmp'
        gff_df_CDS.to_csv(in_dir+'/'+out_fn, sep='\t', header=None, index=None)
        out_fn_2 = out_fn.replace('gff3','fa')
        #this is the bedtools command and should be run in a subproccess or such.
        !bedtools getfasta -s -name -fullHeader -fi {in_dir+'/'+genome_fa[i]} -bed {in_dir+'/'+out_fn} -fo {in_dir+'/'+out_fn_2}

Pst_104E_v12_h_ctg.anno.gff3


/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Pst_104E_v12_p_ctg.anno.gff3
Pst_104E_v12_ph_ctg.anno.gff3


In [31]:
#now pull in the tmp file generated by the bedtools
CDS_dict ={}
CDS_sequences = {}
protein_sequences = {}
non_phase_dict = {}
count = 0
not_multiples = []
#generate a dictionary of CDS sequences. Key being the CDS id and value being the dict key=partial_CDS_ID
#value being the sequence objects
for gff_CDS_tmp in [x for x in os.listdir(in_dir) if x.endswith('CDS.fa.tmp')]:
    print("Working on %s." %(gff_CDS_tmp))
    for record in SeqIO.parse(in_dir+'/'+gff_CDS_tmp, 'fasta'):
        #parsing out the gene/protein identifier
        _id = '.'.join(record.id.split('.')[:-1])  
        #dict key=partial_CDS_ID
        if _id not in CDS_dict.keys():
            CDS_dict[_id] = {record.id:record.seq}
        else:
            CDS_dict[_id].update({record.id:record.seq})

    #loop through the dictonary. Make an ordered dict for each CDS dict. Use a second dict with int(start):key paring.
    #use a order start dict to pull the right order of keys for the CDS sequence dict.
    #Parse out strand and frame. Based on this concatenate the sequences and translate.
    for record_id in CDS_dict.keys():
        tmp_dict = CDS_dict[record_id]
        tmp_start_dict = {}
        for x in tmp_dict.keys():
            #need to get the keys as intergers to sort them numerically
            start = int(x.split('.')[-1].split('_')[0]) #split off the start from the key.
            tmp_start_dict.update({start:x})
        tmp_od_dict = collections.OrderedDict(sorted(tmp_start_dict.items(), key=lambda t:t[0])) #not sure if the lamda is
        #necessary a but it is working.
        sequence =''
        #check if the sequence is reversed using the strand information
        if set([x[-3] for x in tmp_dict.keys()]) == {'-'}:
            for x in reversed(tmp_od_dict):
                key = tmp_od_dict[x]
                sequence = sequence + tmp_dict[key]
                #pull out the frame
                sub_set = [tmp_od_dict[x] for x in reversed(tmp_od_dict)][0][-1]
                #add to non 0 phase dict if necessary
                if sub_set != '0':
                    non_phase_dict.update({record_id:sub_set}) #this is used later to update the main protein dict if frame was not 0.

        elif set([x[-3] for x in tmp_dict.keys()]) == {'+'}: #see above simply + strand
            for x in tmp_od_dict.keys():
                key = tmp_od_dict[x]
                sequence = sequence + tmp_dict[key]
                sub_set = sub_set = [tmp_od_dict[x] for x in tmp_od_dict][0][-1]
                if sub_set != '0':
                    non_phase_dict.update({record_id:sub_set})


        else: #security check
            print('Something with the parsing went wrong!')
            print(record_id)
            #print(sequences)

        if len(sequence)%3 != 0: #some sequences are not //3 = 0
            #print(record_id)
            not_multiples.append(record_id)
        #print(sequences)
        #add all sequences to the dictionaries
        prot_seq = sequence.translate()
        CDS_record = SeqRecord(sequence, id=record_id, description='')
        protein_record = SeqRecord(prot_seq, id=record_id, description='')
        CDS_sequences.update({record_id : CDS_record})
        protein_sequences.update({record_id :protein_record})
    #write complete CDS out to file
    out_fn_CDS = in_dir+'/'+gff_CDS_tmp.replace('.tmp','')
    out_fn_protein = out_fn_CDS.replace("CDS", 'protein2')
    out_hn_CDS = open(out_fn_CDS, 'w')
    SeqIO.write(CDS_sequences.values(), out_hn_CDS, 'fasta')
    out_hn_CDS.close()
    #fix the none 0 frame translations
    for x in non_phase_dict.keys():
        sub_set = int(non_phase_dict[x])
        sequence = CDS_sequences[x].seq
        prot_seq = sequence[sub_set:].translate()
        protein_record = SeqRecord(prot_seq, id=x, description='')
        protein_sequences.update({x :protein_record})
    #write this fix dict out
    out_hn_protein = open(out_fn_protein, 'w')
    SeqIO.write(protein_sequences.values(), out_fn_protein, 'fasta')
    out_hn_protein.close()

Working on Pst_104E_v12_p_ctg.anno.CDS.fa.tmp.


/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/Seq.py:2095: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


Working on Pst_104E_v12_h_ctg.anno.CDS.fa.tmp.
Working on Pst_104E_v12_ph_ctg.anno.CDS.fa.tmp.


In [32]:
#quick check if nothing went wrong with multiple stop codons. 
count = 0
for x in protein_sequences.keys():
    if protein_sequences[x].seq.count('*') > 1:
        print(protein_sequences[x])
        count += 1
print(count)

0


In [33]:
#quick check if nothing went compared to previous file. 
for x in SeqIO.parse(in_dir+'/Pst_104E_v12_ph_ctg.anno.protein.fa', format='fasta'):
    if x.seq != protein_sequences[x.id].seq:
        print(x.id)

evm.model.hcontig_031_015.8


In [36]:
#remove files
for x in [x for x in os.listdir(in_dir) if x.endswith('.tmp')]:
    full_fn = in_dir +'/'+x
    os.remove(full_fn)